# DATA TRANSFORMATION - 2_ACT
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [42]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [43]:
modulo = 'ACT'

In [44]:
dcc = gf.diccionario_llaves()

In [45]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,DS_NOMBRE,NM_PUNTAJE,DS_RESULTADO,fecha,year,month,year_month
0,502989,Prueba de control del Asma ACT,24,NO totalmente controlada,2014-10-28 13:24:49,2014,10,201410
1,502989,Prueba de control del Asma ACT,24,NO totalmente controlada,2014-10-28 13:24:49,2014,10,201410
2,502989,Prueba de control del Asma ACT,24,NO totalmente controlada,2014-10-28 13:24:49,2014,10,201410
3,502989,Prueba de control del Asma ACT,24,NO totalmente controlada,2014-10-28 13:24:49,2014,10,201410
4,502989,Prueba de control del Asma ACT,24,NO totalmente controlada,2014-10-28 13:24:49,2014,10,201410


## 2. Transform

In [46]:
base.rename( columns={ 'DS_NOMBRE':'nombre', 'NM_PUNTAJE':'punt_control', 'DS_RESULTADO':'resultado' }, inplace=True)

In [47]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15090 entries, 0 to 15089
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            15090 non-null  int64         
 1   nombre        15090 non-null  object        
 2   punt_control  15090 non-null  int64         
 3   resultado     15090 non-null  object        
 4   fecha         15090 non-null  datetime64[ns]
 5   year          15090 non-null  int64         
 6   month         15090 non-null  int64         
 7   year_month    15090 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 943.2+ KB


In [48]:
# Cambiar texto a minuscula.
base = gf.letra_lower(base,'nombre')

# Elimina dplicados
base = base.drop_duplicates()

# Elimina columnas sobran
base = base.drop(columns=['nombre','resultado'])


In [49]:
base

,id,punt_control,fecha,year,month,year_month
0,502989,24,2014-10-28 13:24:49,2014,10,201410
10,502989,21,2017-01-03 15:07:43,2017,1,201701
20,502989,25,2018-09-25 16:08:27,2018,9,201809
40,502989,25,2020-04-24 11:54:03,2020,4,202004
50,506808,12,2015-12-05 08:14:06,2015,12,201512
...,...,...,...,...,...,...
15045,1998401,25,2016-10-11 16:39:15,2016,10,201610
15054,1998401,24,2018-09-19 18:15:10,2018,9,201809
15063,1998401,21,2020-04-22 08:52:49,2020,4,202004
15072,1999629,24,2018-05-04 14:43:43,2018,5,201805


In [50]:
base_ = base.groupby(['id','year','month'])['punt_control'].mean().\
    reset_index( name = 'punt_control' )
base_.head()

,id,year,month,punt_control
0,502989,2014,10,24.0
1,502989,2017,1,21.0
2,502989,2018,9,25.0
3,502989,2020,4,25.0
4,506808,2015,12,12.0


In [51]:
# add prefix
pref = 'act_'
base_.columns = [pref + s for s in base_.columns]
base_.rename( columns={ pref+'id':'id', pref+'year':'year', pref+'month':'month' }, inplace=True)
base_.head()

,id,year,month,act_punt_control
0,502989,2014,10,24.0
1,502989,2017,1,21.0
2,502989,2018,9,25.0
3,502989,2020,4,25.0
4,506808,2015,12,12.0


## 3. Merge with ids table

In [52]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_act = ids_mensual.merge( base_, how='left')
base_final_act.head()

,id,year,month,act_punt_control
0,500547,2016,1,NaN
1,500547,2016,2,NaN
2,500547,2016,3,NaN
3,500547,2016,4,NaN
4,500547,2016,5,NaN


In [53]:
base_final_act.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                53280 non-null  int64  
 1   year              53280 non-null  int64  
 2   month             53280 non-null  int64  
 3   act_punt_control  1241 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 2.0 MB
